 # 4.5. Decadencia de peso
COLABORACIÓN [MXNET]Abre la libreta en Colab
Ahora que hemos caracterizado el problema del sobreajuste, podemos introducir algunas técnicas estándar para regularizar modelos. Recuerde que siempre podemos mitigar el sobreajuste saliendo y recopilando más datos de entrenamiento. Eso puede ser costoso, llevar mucho tiempo o estar completamente fuera de nuestro control, haciéndolo imposible a corto plazo. Por ahora, podemos asumir que ya tenemos tantos datos de alta calidad como nuestros recursos lo permitan y centrarnos en las técnicas de regularización.

Recuerde que en nuestro ejemplo de regresión polinomial ( Sección 4.4 ) podríamos limitar la capacidad de nuestro modelo simplemente ajustando el grado del polinomio ajustado. De hecho, limitar el número de características es una técnica popular para mitigar el sobreajuste. Sin embargo, simplemente dejar de lado las características puede ser un instrumento demasiado contundente para el trabajo. Siguiendo con el ejemplo de regresión polinomial, considere lo que podría suceder con entradas de alta dimensión. Las extensiones naturales de polinomios a datos multivariados se denominan monomios , que son simplemente productos de potencias de variables. El grado de un monomio es la suma de las potencias. Por ejemplo, x21x2
 , y x3x25
  ambos son monomios de grado 3.

Nótese que el número de términos con grado d
 explota rápidamente como  d
 crece más grande Dado k
 variables, el número de monomios de grado d
 (es decir, k
 multiselección d
 ) es  (k−1+dk−1)
 . Incluso pequeños cambios en el grado, digamos de 2
 a 3
 , aumentan drásticamente la complejidad de nuestro modelo. Por lo tanto, a menudo necesitamos una herramienta más detallada para ajustar la complejidad de la función.

## 4.5.1. Normas y Decadencia de Peso
Hemos descrito tanto el L2
 norma y la L1
 norma, que son casos especiales de las más generales. Lp
 norma en la Sección 2.3.10 . Pérdida de peso (comúnmente llamada  L2
 regularización), podría ser la técnica más utilizada para regularizar modelos de aprendizaje automático paramétrico. La técnica está motivada por la intuición básica de que entre todas las funciones f
 , la función f=0
 (asignando el valor 0
 a todas las entradas) es en cierto sentido el más simple , y que podemos medir la complejidad de una función por su distancia desde cero. Pero, ¿con qué precisión debemos medir la distancia entre una función y cero? No hay una única respuesta correcta. De hecho, ramas enteras de las matemáticas, incluidas partes del análisis funcional y la teoría de los espacios de Banach, están dedicadas a responder a este problema.

Una interpretación simple podría ser medir la complejidad de una función lineal f(x)=w⊤x
 por alguna norma de su vector de peso, por ejemplo, ∥w∥2
 . El método más común para asegurar un vector de peso pequeño es agregar su norma como término de penalización al problema de minimizar la pérdida. Por lo tanto, reemplazamos nuestro objetivo original, minimizando la pérdida de predicción en las etiquetas de entrenamiento , con un nuevo objetivo, minimizando la suma de la pérdida de predicción y el término de penalización . Ahora, si nuestro vector de peso crece demasiado, nuestro algoritmo de aprendizaje podría enfocarse en minimizar la norma de peso  ∥w∥2
 vs. minimizar el error de entrenamiento. Eso es exactamente lo que queremos. Para ilustrar las cosas en el código, revivamos nuestro ejemplo anterior de la Sección 3.1 para la regresión lineal. Allí, nuestra pérdida fue dada por

(4.5.1)
L(w,b)=1n∑i=1n12(w⊤x(i)+b−y(i))2.
 
Recordar que x(i)
 son las características, y(i)
  son etiquetas para todos los ejemplos de datos i
 , y (w,b)
  son los parámetros de peso y sesgo, respectivamente. Para penalizar el tamaño del vector de peso, debemos agregar de alguna manera ∥w∥2
 a la función de pérdida, pero ¿cómo debería el modelo compensar la pérdida estándar por esta nueva penalización aditiva? En la práctica, caracterizamos esta compensación a través de la constante de regularización  λ
 , un hiperparámetro no negativo que ajustamos usando datos de validación:

(4.5.2)
L(w,b)+λ2∥w∥2,
 
Para λ=0
 , recuperamos nuestra función de pérdida original. Para  λ>0
 , restringimos el tamaño de ∥w∥
 . Dividimos por 2
 por convención: cuando tomamos la derivada de una función cuadrática, la 2
 y 1/2
 cancelar, asegurándose de que la expresión de la actualización se vea agradable y simple. El lector astuto podría preguntarse por qué trabajamos con la norma al cuadrado y no con la norma estándar (es decir, la distancia euclidiana). Hacemos esto por conveniencia computacional. Al cuadrar el L2
 norma, eliminamos la raíz cuadrada, dejando la suma de cuadrados de cada componente del vector de peso. Esto hace que la derivada de la penalización sea fácil de calcular: la suma de las derivadas es igual a la derivada de la suma.

Además, podría preguntarse por qué trabajamos con el L2
 norma en primer lugar y no, digamos, la L1
 norma. De hecho, otras opciones son válidas y populares en las estadísticas. Tiempo  L2
 -los modelos lineales regularizados constituyen el clásico algoritmo de regresión de crestas , L1
 -La regresión lineal regularizada es un modelo igualmente fundamental en estadística, que se conoce popularmente como regresión de lazo .

Una razón para trabajar con el L2
 la norma es que coloca una penalización descomunal en componentes grandes del vector de peso. Esto sesga nuestro algoritmo de aprendizaje hacia modelos que distribuyen el peso de manera uniforme entre una mayor cantidad de funciones. En la práctica, esto podría hacerlos más robustos al error de medición en una sola variable. Por el contrario,  L1
 las penalizaciones conducen a modelos que concentran pesos en un pequeño conjunto de características borrando los otros pesos a cero. Esto se denomina selección de características , que puede ser deseable por otras razones.

Usando la misma notación en (3.1.10) , el descenso de gradiente estocástico del minilote se actualiza para  L2
 -La regresión regularizada sigue:

(4.5.3)
w←(1−ηλ)w−η|B|∑i∈Bx(i)(w⊤x(i)+b−y(i)).
 
Como antes, actualizamos w
 basado en la cantidad por la cual nuestra estimación difiere de la observación. Sin embargo, también reducimos el tamaño de w
 hacia cero. Es por eso que el método a veces se llama "descenso de peso": dado solo el término de penalización, nuestro algoritmo de optimización decae el peso en cada paso del entrenamiento. A diferencia de la selección de funciones, la disminución de peso nos ofrece un mecanismo continuo para ajustar la complejidad de una función. valores menores de  λ
 corresponden a menos restringido w
 , mientras que valores mayores de λ
 constreñir w
  más considerablemente.

Si incluimos una penalización por sesgo correspondiente b2
 puede variar entre implementaciones y puede variar entre capas de una red neuronal. A menudo, no regularizamos el término de sesgo de la capa de salida de una red.

## 4.5.2. Regresión lineal de alta dimensión
Podemos ilustrar los beneficios de la pérdida de peso a través de un ejemplo sintético simple.

In [12]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(max min shuffle);
#use PDL qw(pdl);
use d2l;
use d2l::Timer;
use d2l::Animator;
use d2l::Accumulator;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot;

Primero, generamos algunos datos como antes.

(4.5.4)
y=0.05+∑i=1d0.01xi+ϵ where ϵ∼N(0,0.012).
 
Elegimos que nuestra etiqueta sea una función lineal de nuestras entradas, corrompida por el ruido gaussiano con media cero y desviación estándar de 0,01. Para que los efectos del sobreajuste sean pronunciados, podemos aumentar la dimensionalidad de nuestro problema a d=200
 y trabaje con un pequeño conjunto de entrenamiento que contiene solo 20 ejemplos.

In [23]:
my ($n_train, $n_test, $num_inputs, $batch_size) = [20, 100, 200, 5];
my ($true_w, $true_b) = nd->(($num_inputs, 1)) * (0.01, 0.05);
my $train_data = d2l->synthetic_data($true_w, $true_b, $n_train);
my $train_iter = d2l->load_array($train_data, $batch_size);
my $test_data = d2l->synthetic_data($true_w, $true_b, $n_test);
my $test_iter = d2l->load_array($test_data, $batch_size, $is_train=False);


Warning: Useless use of a constant (0.01) in void context at reply input line 2.


Error: Global symbol "$is_train" requires explicit package name (did you forget to declare "my $is_train"?) at reply input line 6.
BEGIN not safe after errors--compilation aborted at reply input line 11.



## 4.5.3. Implementación desde cero
A continuación, implementaremos la disminución del peso desde cero, simplemente sumando el cuadrado L2
 penalización a la función objetivo original.

### 4.5.3.1. Inicialización de los parámetros del modelo
Primero, definiremos una función para inicializar aleatoriamente los parámetros de nuestro modelo.

In [30]:
sub init_params(){
    my $w = nd->random->normal(scale=1, size=($num_inputs, 1));
    my $b = nd->zeros(1);
    $w->attach_grad();
    $b->attach_grad();

}
    return (w, b);

Error: Can't modify constant item in scalar assignment at reply input line 3, near "1,"
BEGIN not safe after errors--compilation aborted at reply input line 14.



### 4.5.3.2. Definición L2
 Norma Penalización
Quizás la forma más conveniente de implementar esta penalización es elevar al cuadrado todos los términos y resumirlos.

In [31]:
sub l2_penalty($w){

return ($w**2)->sum() / 2
}
   

Warning: Illegal character in prototype for main::l2_penalty : $w at reply input line 1.


Error: Global symbol "$w" requires explicit package name (did you forget to declare "my $w"?) at reply input line 2.
BEGIN not safe after errors--compilation aborted at reply input line 8.



### 4.5.3.3. Definición del ciclo de entrenamiento
El siguiente código ajusta un modelo en el conjunto de entrenamiento y lo evalúa en el conjunto de prueba. La red lineal y la pérdida al cuadrado no han cambiado desde la Sección 3 , por lo que solo las importaremos a través de d2l.linregy d2l.squared_loss. El único cambio aquí es que nuestra pérdida ahora incluye el término de la pena.

In [6]:
sub train(lambd):
    w, b = init_params()
    net, loss = lambda X: d2l.linreg(X, w, b), d2l.squared_loss
    num_epochs, lr = 100, 0.003
    animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log',
                            xlim=[5, num_epochs], legend=['train', 'test'])
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                # The L2 norm penalty term has been added, and broadcasting
                # makes `l2_penalty(w)` a vector whose length is `batch_size`
                l = loss(net(X), y) + lambd * l2_penalty(w)
            l.backward()
            d2l.sgd([w, b], lr, batch_size)
        if (epoch + 1) % 5 == 0:
            animator.add(epoch + 1, (d2l.evaluate_loss(net, train_iter, loss),
                                     d2l.evaluate_loss(net, test_iter, loss)))
    print('L2 norm of w:', np.linalg.norm(w))

Warning: Bareword found where operator expected at reply input line 3, near ")
    net"

	(Missing operator before net?)

Bareword found where operator expected at reply input line 5, near "0.003
    animator"

	(Missing operator before animator?)


Error: syntax error at reply input line 1, near "):"
BEGIN not safe after errors--compilation aborted at reply input line 21.



### 4.5.3.4. Formación sin Regularización
Ahora ejecutamos este código con , deshabilitando el decaimiento de peso. Tenga en cuenta que sobreajustamos mal, disminuyendo el error de entrenamiento pero no el error de prueba, un caso de sobreajuste de libro de texto.lambd = 0

In [7]:
train(lambd=0)

Error: Can't modify constant item in scalar assignment at reply input line 2, near "0)
"
BEGIN not safe after errors--compilation aborted at reply input line 5.



### 4.5.3.5. Uso de la disminución de peso
A continuación, corremos con una disminución sustancial del peso. Tenga en cuenta que el error de entrenamiento aumenta pero el error de prueba disminuye. Este es precisamente el efecto que esperamos de la regularización.

In [8]:
train(lambd=3)

Error: Can't modify constant item in scalar assignment at reply input line 2, near "3)
"
BEGIN not safe after errors--compilation aborted at reply input line 5.



## 4.5.4. Implementación concisa
Debido a que el decaimiento de peso es omnipresente en la optimización de redes neuronales, el marco de aprendizaje profundo lo hace especialmente conveniente, ya que integra el decaimiento de peso en el propio algoritmo de optimización para facilitar su uso en combinación con cualquier función de pérdida. Además, esta integración brinda un beneficio computacional, ya que permite trucos de implementación para agregar disminución de peso al algoritmo, sin ninguna sobrecarga computacional adicional. Dado que la porción de disminución de peso de la actualización depende solo del valor actual de cada parámetro, el optimizador debe tocar cada parámetro una vez de todos modos.

En el siguiente código, especificamos el hiperparámetro de disminución de peso directamente wdal instanciar nuestro Trainer. De forma predeterminada, Gluon decae tanto los pesos como los sesgos simultáneamente. Tenga en cuenta que el hiperparámetro wdse multiplicará por wd_multal actualizar los parámetros del modelo. Por lo tanto, si ponemos wd_multa cero, el parámetro de sesgo b
 no decaerá.

In [9]:
def train_concise(wd):
    net = nn.Sequential()
    net.add(nn.Dense(1))
    net.initialize(init.Normal(sigma=1))
    loss = gluon.loss.L2Loss()
    num_epochs, lr = 100, 0.003
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {
        'learning_rate': lr,
        'wd': wd})
    # The bias parameter has not decayed. Bias names generally end with "bias"
    net.collect_params('.*bias').setattr('wd_mult', 0)
    animator = d2l.Animator(xlabel='epochs', ylabel='loss', yscale='log',
                            xlim=[5, num_epochs], legend=['train', 'test'])
    for epoch in range(num_epochs):
        for X, y in train_iter:
            with autograd.record():
                l = loss(net(X), y)
            l.backward()
            trainer.step(batch_size)
        if (epoch + 1) % 5 == 0:
            animator.add(epoch + 1, (d2l.evaluate_loss(net, train_iter, loss),
                                     d2l.evaluate_loss(net, test_iter, loss)))
    print('L2 norm of w:', np.linalg.norm(net[0].weight.data()))

Warning: Bareword found where operator expected at reply input line 3, near ")
    net"

	(Missing operator before net?)

Bareword found where operator expected at reply input line 4, near ")
    net"

	(Missing operator before net?)

Bareword found where operator expected at reply input line 5, near ")
    loss"

	(Missing operator before loss?)

Bareword found where operator expected at reply input line 6, near ")
    num_epochs"

	(Missing operator before num_epochs?)

Bareword found where operator expected at reply input line 7, near "0.003
    trainer"

	(Missing operator before trainer?)

Bareword found where operator expected at reply input line 11, near ")
    # The bias parameter has not decayed. Bias names generally end with "bias"
    net"

	(Missing operator before net?)

Bareword found where operator expected at reply input line 12, near ")
    animator"

	(Missing operator before animator?)


Error: syntax error at reply input line 1, near "):"
BEGIN not safe after errors--compilation aborted at reply input line 25.



Los gráficos se ven idénticos a aquellos cuando implementamos el decaimiento de peso desde cero. Sin embargo, se ejecutan considerablemente más rápido y son más fáciles de implementar, un beneficio que será más pronunciado para problemas más grandes.

In [10]:
train_concise(0)

Error: Undefined subroutine &main::train_concise called at reply input line 1.


In [11]:
train_concise(3)

Error: Undefined subroutine &main::train_concise called at reply input line 1.


Hasta ahora, solo hemos abordado una noción de lo que constituye una función lineal simple. Además, lo que constituye una función no lineal simple puede ser una cuestión aún más compleja. Por ejemplo, reproducir el espacio de Hilbert del kernel (RKHS) permite aplicar herramientas introducidas para funciones lineales en un contexto no lineal. Desafortunadamente, los algoritmos basados ​​en RKHS tienden a escalar mal a datos grandes y de alta dimensión. En este libro utilizaremos por defecto la simple heurística de aplicar el decaimiento del peso en todas las capas de una red profunda.

## 4.5.5. Resumen
La regularización es un método común para tratar el sobreajuste. Agrega un término de penalización a la función de pérdida en el conjunto de entrenamiento para reducir la complejidad del modelo aprendido.

Una opción particular para mantener el modelo simple es el decaimiento del peso usando un L2
 multa. Esto conduce a una disminución del peso en los pasos de actualización del algoritmo de aprendizaje.

La funcionalidad de disminución de peso se proporciona en optimizadores de marcos de aprendizaje profundo.

Diferentes conjuntos de parámetros pueden tener diferentes comportamientos de actualización dentro del mismo ciclo de entrenamiento.